## SageMaker Pipelines

##### Builds and Trains a Pipeline Model consisting of a preprocessing SKLearn script followed by a Tensorflow model.

##### The pipeline model is then registered to the Model Registry and deployed from there into a real-time endpoint.

### Environment variables

In [2]:
import os
import time
import boto3
import numpy as np
import pandas as pd
import sagemaker
from sagemaker import get_execution_role

In [19]:
session = boto3.Session()
sagemaker_client = session.client("sagemaker")

sagemaker_role = get_execution_role()
print("sagemaker role: ", sagemaker_role)

sagemaker_session = sagemaker.Session(boto_session=session)

bucket_nm = "sagemaker-pipeline-practice"
print("bucket name: ", bucket_nm)

region_nm = session.region_name
print("region name: ", region_nm)

model_package_group_name = "PipelineModelPackageGroup"
prefix = "pipeline-model-example"
pipeline_name = "TrainingPipelineForModel"

sagemaker role:  arn:aws:iam::988889742134:role/service-role/AmazonSageMaker-ExecutionRole-20220315T092490
bucket name:  sagemaker-pipeline-practice
region name:  ap-northeast-2


### Create bucket

In [6]:
!aws s3api create-bucket --bucket $bucket_nm --create-bucket-configuration LocationConstraint=$region_nm

{
    "Location": "http://sagemaker-pipeline-practice.s3.amazonaws.com/"
}


### Download California Housing dataset and upload to Amazon S3

We use the California housing dataset.

More info on the dataset:

This dataset was obtained from the StatLib repository. http://lib.stat.cmu.edu/datasets/

The target variable is the median house value for California districts.

This dataset was derived from the 1990 U.S. census, using one row per census block group. A block group is the smallest geographical unit for which the U.S. Census Bureau publishes sample data (a block group typically has a population of 600 to 3,000 people).

In [10]:
current_path = os.getcwd()
print("current path: ", current_path)

data_dir = os.path.join(current_path, "data")
print("data dir path: ", data_dir)
os.makedirs(data_dir, exist_ok=True)

raw_dir = os.path.join(current_path, "data/raw")
print("raw data dir path: ", raw_dir)
os.makedirs(raw_dir, exist_ok=True)

current path:  /root/amazon-sagemaker-practices/train_register_and_deploy_a_pipeline_model
data dir path:  /root/amazon-sagemaker-practices/train_register_and_deploy_a_pipeline_model/data
raw data dir path:  /root/amazon-sagemaker-practices/train_register_and_deploy_a_pipeline_model/data/raw


In [11]:
!aws s3 cp s3://sagemaker-sample-files/datasets/tabular/california_housing/cal_housing.tgz .

download: s3://sagemaker-sample-files/datasets/tabular/california_housing/cal_housing.tgz to ./cal_housing.tgz


In [12]:
!tar -zxf cal_housing.tgz

tar: CaliforniaHousing/cal_housing.data: Cannot change ownership to uid 10017, gid 166: Operation not permitted
tar: CaliforniaHousing/cal_housing.domain: Cannot change ownership to uid 10017, gid 166: Operation not permitted
tar: Exiting with failure status due to previous errors


In [13]:
columns = [
    "longitude",
    "latitude",
    "housingMedianAge",
    "totalRooms",
    "totalBedrooms",
    "population",
    "households",
    "medianIncome",
    "medianHouseValue",
]

cal_housing_df = pd.read_csv("CaliforniaHousing/cal_housing.data", names=columns, header=None)

In [14]:
cal_housing_df.head()

,longitude,latitude,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


In [15]:
cal_housing_df[
    "medianHouseValue"
] /= 500000  # Scaling target down to avoid overcomplicating the example

In [16]:
cal_housing_df.head()

,longitude,latitude,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,0.9052
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,0.7170
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,0.7042
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,0.6826
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,0.6844


In [17]:
cal_housing_df.to_csv(f"./data/raw/raw_data_all.csv", header=True, index=False)

In [18]:
rawdata_s3_prefix = "{}/data/raw".format(prefix)
print("raw data s3 prefix: ", rawdata_s3_prefix)

raw data s3 prefix:  pipeline-model-example/data/raw


In [22]:
raw_s3 = sagemaker_session.upload_data(
    path="./data/raw/",
    bucket=bucket_nm,
    key_prefix=rawdata_s3_prefix
)
print("input data s3 path: ", raw_s3)

input data s3 path:  s3://sagemaker-pipeline-practice/pipeline-model-example/data/raw


### Define Parameters to Parametrize Pipeline Execution
- Define Pipeline parameters that you can use to parametrize the pipeline.
- Parameters enable custom pipeline executions and schedules without having to modify the Pipeline definition.

The supported parameter types include:
- ParameterString: represents a str Python type
- ParameterInteger: represents an int Python type
- ParameterFloat: represents a float Python type

In [34]:
from sagemaker.workflow.parameters import ParameterInteger, ParameterString, ParameterFloat


# raw input data
input_data_info = {
    "name": "InputData",
    "default_value": raw_s3
}

input_data = ParameterString(**input_data_info)
print(f"{input_data.name} - {input_data.default_value}")

# status of newly trained model in registry
model_approval_status_info = {
    "name": "ModelApprovalStatus",
    "default_value": "Approved"
}

model_approval_status = ParameterString(**model_approval_status_info)
print(f"{model_approval_status.name} - {model_approval_status.default_value}")

# processing step parameters
processing_instance_count = ParameterInteger(
    name="ProcessingInstanceCount", 
    default_value=1
)
print(f"{processing_instance_count.name} - {processing_instance_count.default_value}")

processing_instance_type = ParameterString(
    name="ProcessingInstanceType",
    default_value="ml.m5.large"
)
print(f"{processing_instance_type.name} - {processing_instance_type.default_value}")

# training step parameters
training_instance_type = ParameterString(
    name="TrainingInstanceType", 
    default_value="ml.m5.xlarge"
)
print(f"{training_instance_type.name} - {training_instance_type.default_value}")

training_epochs = ParameterString(
    name="TrainingEpochs", 
    default_value="100"
)
print(f"{training_epochs.name} - {training_epochs.default_value}")

# model performance step parameters
accuracy_mse_threshold = ParameterFloat(
    name="AccuracyMseThreshold", 
    default_value=0.75
)
print(f"{accuracy_mse_threshold.name} - {accuracy_mse_threshold.default_value}")

InputData - s3://sagemaker-pipeline-practice/pipeline-model-example/data/raw
ModelApprovalStatus - Approved
ProcessingInstanceCount - 1
ProcessingInstanceType - ml.m5.large
TrainingInstanceType - ml.m5.xlarge
TrainingEpochs - 100
AccuracyMseThreshold - 0.75


### Define a Processing Step for Feature Engineering

In [35]:
!mkdir -p code

In [36]:
%%writefile code/preprocess.py

import glob
import numpy as np
import pandas as pd
import os
import json
import joblib
from io import StringIO
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tarfile

try:
    from sagemaker_containers.beta.framework import (
        content_types,
        encoders,
        env,
        modules,
        transformer,
        worker,
        server,
    )
except ImportError:
    pass

feature_columns = [
    "longitude",
    "latitude",
    "housingMedianAge",
    "totalRooms",
    "totalBedrooms",
    "population",
    "households",
    "medianIncome",
]
label_column = "medianHouseValue"

base_dir = "/opt/ml/processing"
base_output_dir = "/opt/ml/output/"

if __name__ == "__main__":
    df = pd.read_csv(f"{base_dir}/input/raw_data_all.csv")
    feature_data = df.drop(label_column, axis=1, inplace=False)
    label_data = df[label_column]
    x_train, x_test, y_train, y_test = train_test_split(feature_data, label_data, test_size=0.33)

    scaler = StandardScaler()

    scaler.fit(x_train)
    x_train = scaler.transform(x_train)
    x_test = scaler.transform(x_test)

    train_dataset = pd.concat([pd.DataFrame(x_train), y_train.reset_index(drop=True)], axis=1)
    test_dataset = pd.concat([pd.DataFrame(x_test), y_test.reset_index(drop=True)], axis=1)

    train_dataset.columns = feature_columns + [label_column]
    test_dataset.columns = feature_columns + [label_column]

    train_dataset.to_csv(f"{base_dir}/train/train.csv", header=True, index=False)
    test_dataset.to_csv(f"{base_dir}/test/test.csv", header=True, index=False)
    joblib.dump(scaler, "model.joblib")
    with tarfile.open(f"{base_dir}/scaler_model/model.tar.gz", "w:gz") as tar_handle:
        tar_handle.add(f"model.joblib")


def input_fn(input_data, content_type):
    """Parse input data payload

    We currently only take csv input. Since we need to process both labelled
    and unlabelled data we first determine whether the label column is present
    by looking at how many columns were provided.
    """
    if content_type == "text/csv":
        # Read the raw input data as CSV.
        df = pd.read_csv(StringIO(input_data), header=None)

        if len(df.columns) == len(feature_columns) + 1:
            # This is a labelled example, includes the ring label
            df.columns = feature_columns + [label_column]
        elif len(df.columns) == len(feature_columns):
            # This is an unlabelled example.
            df.columns = feature_columns

        return df
    else:
        raise ValueError("{} not supported by script!".format(content_type))


def output_fn(prediction, accept):
    """Format prediction output

    The default accept/content-type between containers for serial inference is JSON.
    We also want to set the ContentType or mimetype as the same value as accept so the next
    container can read the response payload correctly.
    """
    if accept == "application/json":
        instances = []
        for row in prediction.tolist():
            instances.append(row)
        json_output = {"instances": instances}

        return worker.Response(json.dumps(json_output), mimetype=accept)
    elif accept == "text/csv":
        return worker.Response(encoders.encode(prediction, accept), mimetype=accept)
    else:
        raise RuntimeException("{} accept type is not supported by this script.".format(accept))


def predict_fn(input_data, model):
    """Preprocess input data

    We implement this because the default predict_fn uses .predict(), but our model is a preprocessor
    so we want to use .transform().

    The output is returned in the following order:

        rest of features either one hot encoded or standardized
    """
    features = model.transform(input_data)

    if label_column in input_data:
        # Return the label (as the first column) and the set of features.
        return np.insert(features, 0, input_data[label_column], axis=1)
    else:
        # Return only the set of features
        return features


def model_fn(model_dir):
    """Deserialize fitted model"""
    preprocessor = joblib.load(os.path.join(model_dir, "model.joblib"))
    return preprocessor

Writing code/preprocess.py


#### Define SKLearnProcessor

In [37]:
from sagemaker.sklearn.processing import SKLearnProcessor


sklearn_framework_version = "0.23-1"

sklearn_processor = SKLearnProcessor(
    framework_version=sklearn_framework_version,
    instance_type=processing_instance_type,
    instance_count=processing_instance_count,
    base_job_name="sklearn-housing-data-process",
    role=sagemaker_role,
)

#### Define step_process for pipeline

In [38]:
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep


step_process = ProcessingStep(
    name="PreprocessData",
    processor=sklearn_processor,
    inputs=[
        ProcessingInput(source=input_data, destination="/opt/ml/processing/input"),
    ],
    outputs=[
        ProcessingOutput(output_name="scaler_model", source="/opt/ml/processing/scaler_model"),
        ProcessingOutput(output_name="train", source="/opt/ml/processing/train"),
        ProcessingOutput(output_name="test", source="/opt/ml/processing/test"),
    ],
    code="code/preprocess.py",
)

### Define a Training Step to Train a Model

In [39]:
%%writefile code/train.py

import argparse
import numpy as np
import os
import tensorflow as tf
import pandas as pd

feature_columns = [
    "longitude",
    "latitude",
    "housingMedianAge",
    "totalRooms",
    "totalBedrooms",
    "population",
    "households",
    "medianIncome",
]
label_column = "medianHouseValue"


def parse_args():

    parser = argparse.ArgumentParser()

    # hyperparameters sent by the client are passed as command-line arguments to the script
    parser.add_argument("--epochs", type=int, default=1)
    parser.add_argument("--batch_size", type=int, default=64)
    parser.add_argument("--learning_rate", type=float, default=0.1)

    # data directories
    parser.add_argument("--train", type=str, default=os.environ.get("SM_CHANNEL_TRAIN"))
    parser.add_argument("--test", type=str, default=os.environ.get("SM_CHANNEL_TEST"))

    # model directory
    parser.add_argument("--sm-model-dir", type=str, default=os.environ.get("SM_MODEL_DIR"))

    return parser.parse_known_args()


def get_train_data(train_dir):
    train_data = pd.read_csv(os.path.join(train_dir, "train.csv"))
    x_train = train_data[feature_columns].to_numpy()
    y_train = train_data[label_column].to_numpy()
    print("x train", x_train.shape, "y train", y_train.shape)

    return x_train, y_train


def get_test_data(test_dir):

    test_data = pd.read_csv(os.path.join(test_dir, "test.csv"))
    x_test = test_data[feature_columns].to_numpy()
    y_test = test_data[label_column].to_numpy()
    print("x test", x_test.shape, "y test", y_test.shape)

    return x_test, y_test


def get_model():

    inputs = tf.keras.Input(shape=(8,))
    hidden_1 = tf.keras.layers.Dense(8, activation="tanh")(inputs)
    hidden_2 = tf.keras.layers.Dense(4, activation="sigmoid")(hidden_1)
    outputs = tf.keras.layers.Dense(1)(hidden_2)
    return tf.keras.Model(inputs=inputs, outputs=outputs)


if __name__ == "__main__":

    args, _ = parse_args()

    print("Training data location: {}".format(args.train))
    print("Test data location: {}".format(args.test))
    x_train, y_train = get_train_data(args.train)
    x_test, y_test = get_test_data(args.test)

    batch_size = args.batch_size
    epochs = args.epochs
    learning_rate = args.learning_rate
    print(
        "batch_size = {}, epochs = {}, learning rate = {}".format(batch_size, epochs, learning_rate)
    )

    model = get_model()
    optimizer = tf.keras.optimizers.SGD(learning_rate)
    model.compile(optimizer=optimizer, loss="mse")
    model.fit(
        x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test, y_test)
    )

    # evaluate on test set
    scores = model.evaluate(x_test, y_test, batch_size, verbose=2)
    print("\nTest MSE :", scores)

    # save model
    model.save(args.sm_model_dir + "/1")

Writing code/train.py


#### Define tensorflow estimator

In [40]:
from sagemaker.tensorflow import TensorFlow
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TrainingStep
from sagemaker.workflow.step_collections import RegisterModel
import time

# Where to store the trained model
model_path = f"s3://{bucket_nm}/{prefix}/model/"
print("model path: ", model_path)

hyperparameters = {"epochs": training_epochs}
tensorflow_version = "2.4.1"
python_version = "py37"

tf2_estimator = TensorFlow(
    source_dir="code",
    entry_point="train.py",
    instance_type=training_instance_type,
    instance_count=1,
    framework_version=tensorflow_version,
    role=sagemaker_role,
    base_job_name="tensorflow-train-model",
    output_path=model_path,
    hyperparameters=hyperparameters,
    py_version=python_version,
)

model path:  s3://sagemaker-pipeline-practice/pipeline-model-example/model/


### Define step_train_model for pipeline

In [41]:
# Use the tf2_estimator in a Sagemaker pipelines ProcessingStep.
# NOTE how the input to the training job directly references the output of the previous step.
step_train_model = TrainingStep(
    name="TrainTensorflowModel",
    estimator=tf2_estimator,
    inputs={
        "train": TrainingInput(
            s3_data=step_process.properties.ProcessingOutputConfig.Outputs["train"].S3Output.S3Uri,
            content_type="text/csv",
        ),
        "test": TrainingInput(
            s3_data=step_process.properties.ProcessingOutputConfig.Outputs["test"].S3Output.S3Uri,
            content_type="text/csv",
        ),
    },
)

### Define a Model Evaluation Step to Evaluate the Trained Model

In [42]:
%%writefile code/evaluate.py

import os
import json
import sys
import numpy as np
import pandas as pd
import pathlib
import tarfile


feature_columns = [
    "longitude",
    "latitude",
    "housingMedianAge",
    "totalRooms",
    "totalBedrooms",
    "population",
    "households",
    "medianIncome",
]
label_column = "medianHouseValue"

if __name__ == "__main__":

    model_path = f"/opt/ml/processing/model/model.tar.gz"
    with tarfile.open(model_path, "r:gz") as tar:
        tar.extractall("./model")
    import tensorflow as tf

    model = tf.keras.models.load_model("./model/1")
    test_path = "/opt/ml/processing/test/"
    df = pd.read_csv(test_path + "/test.csv")
    x_test = df[feature_columns].to_numpy()
    y_test = df[label_column].to_numpy()
    scores = model.evaluate(x_test, y_test, verbose=2)
    print("\nTest MSE :", scores)

    # Available metrics to add to model: https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-model-quality-metrics.html
    report_dict = {
        "regression_metrics": {
            "mse": {"value": scores, "standard_deviation": "NaN"},
        },
    }

    output_dir = "/opt/ml/processing/evaluation"
    pathlib.Path(output_dir).mkdir(parents=True, exist_ok=True)

    evaluation_path = f"{output_dir}/evaluation.json"
    with open(evaluation_path, "w") as f:
        f.write(json.dumps(report_dict))

Writing code/evaluate.py


#### Define evaluation docker image uri & evaluation model processor

In [44]:
from sagemaker.workflow.properties import PropertyFile
from sagemaker.sklearn.processing import ScriptProcessor


tf_eval_image_uri = sagemaker.image_uris.retrieve(
    framework="tensorflow",
    region=region_nm,
    version=tensorflow_version,
    image_scope="training",
    py_version="py37",
    instance_type=training_instance_type,
)

evaluate_model_processor = ScriptProcessor(
    role=sagemaker_role,
    image_uri=tf_eval_image_uri,
    command=["python3"],
    instance_count=1,
    instance_type=training_instance_type,
)

# Create a PropertyFile
# A PropertyFile is used to be able to reference outputs from a processing step, for instance to use in a condition step.
# For more information, visit https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-propertyfile.html
evaluation_report = PropertyFile(
    name="EvaluationReport", output_name="evaluation", path="evaluation.json"
)

#### Define step_evaluate_model for pipeline

In [45]:
# Use the evaluate_model_processor in a Sagemaker pipelines ProcessingStep.
step_evaluate_model = ProcessingStep(
    name="EvaluateModelPerformance",
    processor=evaluate_model_processor,
    inputs=[
        ProcessingInput(
            source=step_train_model.properties.ModelArtifacts.S3ModelArtifacts,
            destination="/opt/ml/processing/model",
        ),
        ProcessingInput(
            source=step_process.properties.ProcessingOutputConfig.Outputs["test"].S3Output.S3Uri,
            destination="/opt/ml/processing/test",
        ),
    ],
    outputs=[
        ProcessingOutput(output_name="evaluation", source="/opt/ml/processing/evaluation"),
    ],
    code="code/evaluate.py",
    property_files=[evaluation_report],
)

### Define a Register Model Step to Create a Model Package for the PipelineModel

In [47]:
from sagemaker.model import Model
from sagemaker.sklearn.model import SKLearnModel
from sagemaker import PipelineModel


# scaler model - preprocess
scaler_model_s3 = "{}/model.tar.gz".format(
    step_process.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"]
)

scaler_model = SKLearnModel(
    model_data=scaler_model_s3,
    role=sagemaker_role,
    sagemaker_session=sagemaker_session,
    entry_point="code/preprocess.py",
    framework_version=sklearn_framework_version,
)

# inference model using training model artifact
tf_model_image_uri = sagemaker.image_uris.retrieve(
    framework="tensorflow",
    region=region_nm,
    version=tensorflow_version,
    image_scope="inference",
    py_version="py37",
    instance_type=training_instance_type,
)

tf_model = Model(
    image_uri=tf_model_image_uri,
    model_data=step_train_model.properties.ModelArtifacts.S3ModelArtifacts,
    sagemaker_session=sagemaker_session,
    role=sagemaker_role,
)

# A pipeline of SageMaker Model instances
# This pipeline can be deployed as an Endpoint on SageMaker
# Initialize a SageMaker Model instance
pipeline_model = PipelineModel(
    models=[scaler_model, tf_model], 
    role=sagemaker_role, 
    sagemaker_session=sagemaker_session
)

#### Define step_register_pipeline_model for pipeline

In [48]:
from sagemaker.model_metrics import MetricsSource, ModelMetrics
from sagemaker.workflow.step_collections import RegisterModel


evaluation_s3_uri = "{}/evaluation.json".format(
    step_evaluate_model.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"]
)

model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri=evaluation_s3_uri,
        content_type="application/json",
    )
)

step_register_pipeline_model = RegisterModel(
    name="PipelineModel",
    model=pipeline_model,
    content_types=["text/csv"],
    response_types=["text/csv"],
    inference_instances=["ml.m5.large", "ml.m5.xlarge"],
    transform_instances=["ml.m5.xlarge"],
    model_package_group_name=model_package_group_name,
    model_metrics=model_metrics,
    approval_status=model_approval_status,
)

### Define a Condition Step to Check Accuracy and Conditionally Register a Model in the Model Registry

In [49]:
from sagemaker.workflow.conditions import ConditionLessThanOrEqualTo
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.functions import JsonGet

# Create accuracy condition to ensure the model meets performance requirements.
# Models with a test accuracy lower than the condition will not be registered with the model registry.
cond_lte = ConditionLessThanOrEqualTo(
    left=JsonGet(
        step_name=step_evaluate_model.name,
        property_file=evaluation_report,
        json_path="regression_metrics.mse.value",
    ),
    right=accuracy_mse_threshold,
)

# Create a Sagemaker Pipelines ConditionStep, using the condition above.
# Enter the steps to perform if the condition returns True / False.
step_cond = ConditionStep(
    name="MSE-Lower-Than-Threshold-Condition",
    conditions=[cond_lte],
    if_steps=[step_register_pipeline_model],  # step_register_model, step_register_scaler,
    else_steps=[],
)

### Define a Pipeline of Parameters, Steps and Conditions

In [50]:
from sagemaker.workflow.pipeline import Pipeline

# Create a Sagemaker Pipeline.
# Each parameter for the pipeline must be set as a parameter explicitly when the pipeline is created.
# Also pass in each of the steps created above.
# Note that the order of execution is determined from each step's dependencies on other steps,
# not on the order they are passed in below.
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        processing_instance_type,
        processing_instance_count,
        training_instance_type,
        input_data,
        model_approval_status,
        training_epochs,
        accuracy_mse_threshold,
    ],
    steps=[step_process, step_train_model, step_evaluate_model, step_cond],
)

In [51]:
import json

definition = json.loads(pipeline.definition())
definition

{'Version': '2020-12-01',
 'Metadata': {},
 'Parameters': [{'Name': 'ProcessingInstanceType',
   'Type': 'String',
   'DefaultValue': 'ml.m5.large'},
  {'Name': 'ProcessingInstanceCount', 'Type': 'Integer', 'DefaultValue': 1},
  {'Name': 'TrainingInstanceType',
   'Type': 'String',
   'DefaultValue': 'ml.m5.xlarge'},
  {'Name': 'InputData',
   'Type': 'String',
   'DefaultValue': 's3://sagemaker-pipeline-practice/pipeline-model-example/data/raw'},
  {'Name': 'ModelApprovalStatus',
   'Type': 'String',
   'DefaultValue': 'Approved'},
  {'Name': 'TrainingEpochs', 'Type': 'String', 'DefaultValue': '100'},
  {'Name': 'AccuracyMseThreshold', 'Type': 'Float', 'DefaultValue': 0.75}],
 'PipelineExperimentConfig': {'ExperimentName': {'Get': 'Execution.PipelineName'},
  'TrialName': {'Get': 'Execution.PipelineExecutionId'}},
 'Steps': [{'Name': 'PreprocessData',
   'Type': 'Processing',
   'Arguments': {'ProcessingResources': {'ClusterConfig': {'InstanceType': {'Get': 'Parameters.ProcessingInsta

### Submit the pipeline to SageMaker and start execution

In [52]:
pipeline.upsert(role_arn=sagemaker_role)

{'PipelineArn': 'arn:aws:sagemaker:ap-northeast-2:988889742134:pipeline/trainingpipelineformodel',
 'ResponseMetadata': {'RequestId': 'ea8474af-0624-4f5c-9368-fd124e082fb1',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'ea8474af-0624-4f5c-9368-fd124e082fb1',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '97',
   'date': 'Thu, 14 Apr 2022 12:47:02 GMT'},
  'RetryAttempts': 0}}

In [53]:
execution = pipeline.start()

In [54]:
execution.wait()

### Deploy latest approved model to a real-time endpoint

In [56]:
import argparse
import boto3
import logging
import os
from botocore.exceptions import ClientError
import tarfile
import zipfile


logger = logging.getLogger(__name__)
sm_client = boto3.client("sagemaker")

response = sm_client.list_model_packages(
    ModelPackageGroupName=model_package_group_name,
    ModelApprovalStatus="Approved",
    SortBy="CreationTime",
    MaxResults=100
)

approved_packages = response["ModelPackageSummaryList"]
print("Approved packages: ", approved_packages)

Approved packages:  [{'ModelPackageGroupName': 'PipelineModelPackageGroup', 'ModelPackageVersion': 2, 'ModelPackageArn': 'arn:aws:sagemaker:ap-northeast-2:988889742134:model-package/pipelinemodelpackagegroup/2', 'CreationTime': datetime.datetime(2022, 4, 14, 13, 5, 20, 770000, tzinfo=tzlocal()), 'ModelPackageStatus': 'Completed', 'ModelApprovalStatus': 'Approved'}, {'ModelPackageGroupName': 'PipelineModelPackageGroup', 'ModelPackageVersion': 1, 'ModelPackageArn': 'arn:aws:sagemaker:ap-northeast-2:988889742134:model-package/pipelinemodelpackagegroup/1', 'CreationTime': datetime.datetime(2022, 4, 12, 2, 59, 3, 408000, tzinfo=tzlocal()), 'ModelPackageStatus': 'Completed', 'ModelApprovalStatus': 'Approved'}]


In [61]:
%%writefile utils.py
import argparse
import boto3
import logging
import os
from botocore.exceptions import ClientError
import tarfile
import zipfile

logger = logging.getLogger(__name__)
sm_client = boto3.client("sagemaker")


def get_approved_package(model_package_group_name):
    """Gets the latest approved model package for a model package group.

    Args:
        model_package_group_name: The model package group name.

    Returns:
        The SageMaker Model Package ARN.
    """
    try:
        # Get the latest approved model package
        response = sm_client.list_model_packages(
            ModelPackageGroupName=model_package_group_name,
            ModelApprovalStatus="Approved",
            SortBy="CreationTime",
            MaxResults=100,
        )
        approved_packages = response["ModelPackageSummaryList"]

        # Fetch more packages if none returned with continuation token
        while len(approved_packages) == 0 and "NextToken" in response:
            logger.debug("Getting more packages for token: {}".format(response["NextToken"]))
            response = sm_client.list_model_packages(
                ModelPackageGroupName=model_package_group_name,
                ModelApprovalStatus="Approved",
                SortBy="CreationTime",
                MaxResults=100,
                NextToken=response["NextToken"],
            )
            approved_packages.extend(response["ModelPackageSummaryList"])

        # Return error if no packages found
        if len(approved_packages) == 0:
            error_message = (
                f"No approved ModelPackage found for ModelPackageGroup: {model_package_group_name}"
            )
            logger.error(error_message)
            raise Exception(error_message)

        # Return the pmodel package arn
        model_package_arn = approved_packages[0]["ModelPackageArn"]
        logger.info(f"Identified the latest approved model package: {model_package_arn}")
        return approved_packages[0]
        # return model_package_arn
    except ClientError as e:
        error_message = e.response["Error"]["Message"]
        logger.error(error_message)
        raise Exception(error_message)

Writing utils.py


In [63]:
from utils import get_approved_package


sm_client = boto3.client("sagemaker")

pck = get_approved_package(
    model_package_group_name
)

model_description = sm_client.describe_model_package(
    ModelPackageName=pck["ModelPackageArn"]
)

model_description

{'ModelPackageGroupName': 'PipelineModelPackageGroup',
 'ModelPackageVersion': 2,
 'ModelPackageArn': 'arn:aws:sagemaker:ap-northeast-2:988889742134:model-package/pipelinemodelpackagegroup/2',
 'CreationTime': datetime.datetime(2022, 4, 14, 13, 5, 20, 770000, tzinfo=tzlocal()),
 'InferenceSpecification': {'Containers': [{'Image': '366743142698.dkr.ecr.ap-northeast-2.amazonaws.com/sagemaker-scikit-learn:0.23-1-cpu-py3',
    'ImageDigest': 'sha256:1b1a557ceb336c12d1f8ade5f0cd79781a8df07dde3128544dfef6679feb261e',
    'ModelDataUrl': 's3://sagemaker-ap-northeast-2-988889742134/pipelines-6d2jqq93h7sn-sklearnRepackModel-PYH9GtrZGW/output/model.tar.gz',
    'Environment': {'SAGEMAKER_CONTAINER_LOG_LEVEL': '20',
     'SAGEMAKER_PROGRAM': 'preprocess.py',
     'SAGEMAKER_REGION': 'ap-northeast-2',
     'SAGEMAKER_SUBMIT_DIRECTORY': 's3://sagemaker-ap-northeast-2-988889742134/sagemaker-scikit-learn-2022-04-14-12-43-48-276/sourcedir.tar.gz'}},
   {'Image': '763104351884.dkr.ecr.ap-northeast-2.am

In [65]:
from sagemaker import ModelPackage


model_package_arn = model_description["ModelPackageArn"]

model = ModelPackage(
    role=sagemaker_role,
    model_package_arn=model_package_arn,
    sagemaker_session=sagemaker_session
)

### Create endpoint to deploy the model package

In [66]:
endpoint_name = "DEMO-endpoint-" + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
print("EndpointName= {}".format(endpoint_name))

EndpointName= DEMO-endpoint-2022-04-14-13-41-00


In [67]:
model.deploy(
    initial_instance_count=1, 
    instance_type="ml.m5.xlarge", 
    endpoint_name=endpoint_name
)

-------------!

### Prediction

In [68]:
from sagemaker.predictor import Predictor

predictor = Predictor(endpoint_name=endpoint_name)

In [69]:
data = pd.read_csv("data/raw/raw_data_all.csv")
house_values = data["medianHouseValue"]
data = data.drop("medianHouseValue", axis=1)

In [70]:
data.head()

,longitude,latitude,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462


In [71]:
pred_count = 10
payload = data.iloc[:pred_count].to_string(header=False, index=False).replace("  ", ",")

In [72]:
payload

'-122.23,37.88,41.0, 880.0, 129.0, 322.0, 126.0,8.3252\n-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014\n-122.24,37.85,52.0,1467.0, 190.0, 496.0, 177.0,7.2574\n-122.25,37.85,52.0,1274.0, 235.0, 558.0, 219.0,5.6431\n-122.25,37.85,52.0,1627.0, 280.0, 565.0, 259.0,3.8462\n-122.25,37.85,52.0, 919.0, 213.0, 413.0, 193.0,4.0368\n-122.25,37.84,52.0,2535.0, 489.0,1094.0, 514.0,3.6591\n-122.25,37.84,52.0,3104.0, 687.0,1157.0, 647.0,3.1200\n-122.26,37.84,42.0,2555.0, 665.0,1206.0, 595.0,2.0804\n-122.25,37.84,52.0,3549.0, 707.0,1551.0, 714.0,3.6912'

In [73]:
p = predictor.predict(payload, initial_args={"ContentType": "text/csv"})
print(p.decode("utf-8"))

{
    "predictions": [[0.820725203], [1.00937867], [0.807161331], [0.703612208], [0.518589854], [0.539785266], [0.561942577], [0.620703876], [0.43478477], [0.596874774]
    ]
}


In [74]:
blue, stop = "\033[94m", "\033[0m"
predictions = json.loads(p.decode("utf-8"))["predictions"]
for i in range(pred_count):
    print(
        f"Predicted: {blue}{predictions[i][0]}{stop} and Actual is: {blue}{house_values.iloc[i]}{stop}"
    )

Predicted: 0.820725203 and Actual is: 0.9052
Predicted: 1.00937867 and Actual is: 0.7170000000000001
Predicted: 0.807161331 and Actual is: 0.7042
Predicted: 0.703612208 and Actual is: 0.6826
Predicted: 0.518589854 and Actual is: 0.6844
Predicted: 0.539785266 and Actual is: 0.5394
Predicted: 0.561942577 and Actual is: 0.5984
Predicted: 0.620703876 and Actual is: 0.4828
Predicted: 0.43478477 and Actual is: 0.4534
Predicted: 0.596874774 and Actual is: 0.5222


### Clean-up

In [77]:
sm_client = boto3.client("sagemaker")

for d in sm_client.list_model_packages(
    ModelPackageGroupName=model_package_group_name
)["ModelPackageSummaryList"]:
    print(d["ModelPackageArn"])
#     sm_client.delete_model_package(ModelPackageName=d["ModelPackageArn"])

arn:aws:sagemaker:ap-northeast-2:988889742134:model-package/pipelinemodelpackagegroup/2
arn:aws:sagemaker:ap-northeast-2:988889742134:model-package/pipelinemodelpackagegroup/1


In [78]:
predictor.delete_endpoint()

In [79]:
# pipeline.delete()